# VacationPy
----

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import gkey

### Store Part I results into DataFrame

In [14]:
# load dataframe from WeatherPy and display
filepath = "../WeatherPy/Output/weather_output.csv"
weather_data = pd.read_csv(filepath)
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,busselton,-33.6500,115.3333,51.04,80,93,8.55,au,1626875931
1,punta arenas,-53.1500,-70.9167,35.71,93,75,3.44,cl,1626875914
2,mahebourg,-20.4081,57.7000,70.09,83,75,18.41,mu,1626875954
3,mackay,-21.1500,149.2000,60.04,87,0,5.75,au,1626875954
4,homer,59.6425,-151.5483,55.08,90,100,1.99,us,1626875955
...,...,...,...,...,...,...,...,...,...
557,izhma,65.0083,53.9113,63.70,62,88,11.21,ru,1626876868
558,rawah,34.4816,41.9093,107.33,12,0,16.40,iq,1626876869
559,jawhar,2.7809,45.5005,82.92,59,61,23.47,so,1626876869
560,astoria,46.1879,-123.8313,57.85,94,1,3.44,us,1626876870


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [41]:
# configure gmaps
gmaps.configure(api_key=gkey)

# create figure layout options in dictionary
figure_layout = {
    "width": "850px",
    "height": "500px",
    "border": "5px solid black",
    "padding": "1px"
}

# create variables for heatmap - to be weighted by humidity & max intensity will be max humidity
locations = weather_data[["Lat","Lng"]]
weight = weather_data["Humidity"]
max_humidity = weather_data["Humidity"].max()

# create and display figure with heatmap
fig = gmaps.figure(layout=figure_layout, zoom_level=1.7, center=(20,10))
heatmap_layer = gmaps.heatmap_layer(locations, weights=weight, max_intensity=max_humidity, point_radius=15)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(border='5px solid black', height='500px', padding='1px', width='850px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [48]:
# create ideal new dataframe containing only cities with max temp between 72F & 80F, 
# cloudiness less than 10%, and Humidity less than 50%
vacation_data = weather_data.loc[(weather_data["Max Temp"]>=72) & 
                                 (weather_data["Max Temp"]<=80) & 
                                 (weather_data["Cloudiness"]<10) &
                                 (weather_data["Humidity"]<50), :]

# display dataframe
vacation_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,lavumisa,-27.3100,31.8920,76.84,20,0,7.63,sz,1626875956
68,mvuma,-19.2792,30.5283,73.18,20,1,9.53,zw,1626876382
127,aklavik,68.2191,-135.0107,73.65,45,9,2.13,ca,1626876436
295,trindade,-16.6494,-49.4889,76.75,29,0,7.27,br,1626876615
331,baiyin,36.5583,104.2081,79.50,38,0,8.10,cn,1626876655
400,jatai,-17.8814,-51.7144,78.93,14,0,5.44,br,1626876635
427,atbasar,51.8000,68.3333,79.03,41,8,6.17,kz,1626876741
435,otavi,-19.6500,17.3333,74.48,14,0,7.16,na,1626876749
454,tobane,-21.9500,27.9000,75.60,14,0,9.40,bw,1626876766
498,abadiania,-16.2042,-48.7069,77.95,35,1,8.93,br,1626876815


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [74]:
hotel_data = vacation_data.loc[:,["City","Lat","Lng","Country"]]
hotel_data["Hotel Name"] = None

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {
    "keyword": "hotel",
    "type": "lodging",
    "key": gkey,
    "radius": 5000,
    "location": ""
}

for index, row in hotel_data.iterrows():
    params["location"] = str(row[1]) +","+ str(row[2])
    
    response = requests.get(base_url, params).json()
    
    hotel_results = response["results"]
    
    try:
        print(f"{row['City']}, {row['Country'].upper()} hotel: {hotel_results[0]['name']}")
        hotel_data.loc[index, "Hotel Name"] = hotel_results[0]["name"]
    except (KeyError, IndexError):
        print(f"No hotel found for this city within 5000 meters.")
                                                                        
hotel_data = hotel_data.dropna(how="any")
hotel_data

lavumisa,  SZ hotel: ROADSIDE B&B
mvuma,  ZW hotel: Nyangombe Backpackers
aklavik,  CA hotel: Sundog Inn
trindade,  BR hotel: Hotel Liguori
baiyin,  CN hotel: 7 Days Inn
jatai,  BR hotel: ibis Jatai
atbasar,  KZ hotel: The Ritz-Carlton, Astana
otavi,  NA hotel: Stop 'n Stay Otavi Rest Camp
No hotel found for this city within 5000 meters.
abadiania,  BR hotel: Hotel São Raphael


,City,Lat,Lng,Country,Hotel Name
6,lavumisa,-27.3100,31.8920,sz,ROADSIDE B&B
68,mvuma,-19.2792,30.5283,zw,Nyangombe Backpackers
127,aklavik,68.2191,-135.0107,ca,Sundog Inn
295,trindade,-16.6494,-49.4889,br,Hotel Liguori
331,baiyin,36.5583,104.2081,cn,7 Days Inn
400,jatai,-17.8814,-51.7144,br,ibis Jatai
427,atbasar,51.8000,68.3333,kz,"The Ritz-Carlton, Astana"
435,otavi,-19.6500,17.3333,na,Stop 'n Stay Otavi Rest Camp
498,abadiania,-16.2042,-48.7069,br,Hotel São Raphael


In [77]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_data.iterrows()]
marker_locations = hotel_data[["Lat", "Lng"]]

In [79]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(marker_locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(border='5px solid black', height='500px', padding='1px', width='850px'))